In [ ]:
import cudf
import dask
import numpy as np
import os
import pandas as pd
import dask_cuda
import dask_cudf

## Benchmarks
https://h2oai.github.io/db-benchmark/

https://github.com/Rdatatable/data.table/wiki/Benchmarks-:-Grouping#code-to-reproduce-the-timings-above-

In [ ]:
def randChar(f, numGrp, N):
   things = [f.format(x) for x in range(numGrp)]
   return [things[x] for x in np.random.choice(numGrp, N)]

In [ ]:
def randFloat(numGrp, N) :
   things = [round(100*np.random.random(),4) for x in range(numGrp)]
   return [things[x] for x in np.random.choice(numGrp, N)]

In [ ]:
N = int(1e7)
K = 100

In [ ]:
N // K

In [ ]:
pdf = pd.DataFrame({
  'id1' : randChar("id{0:0=3d}", K, N),       # large groups (char)
  'id2' : randChar("id{0:0=3d}", K, N),       # large groups (char)
  'id3' : randChar("id{0:0=3d}", N//K, N),   # small groups (char)
  'id4' : np.random.choice(K, N),         # large groups (int)
  'id5' : np.random.choice(K, N),         # large groups (int)
  'id6' : np.random.choice(N//K, N),      # small groups (int)
  'v1' :  np.random.choice(5, N),         # int in range [1,5]
  'v2' :  np.random.choice(5, N),         # int in range [1,5]
  'v3' :  randFloat(100,N)                # numeric e.g. 23.5749
})

In [ ]:
cdf = cudf.DataFrame.from_pandas(pdf)

In [ ]:
cdf

In [ ]:
pdf.head()

In [ ]:
ddf = dask.dataframe.from_pandas(pdf, npartitions=8)

In [ ]:
ddf.groupby(["id1", "id2"]).agg({'v1': 'sum'}).compute()

Becnhmark 1

In [ ]:
%%timeit -r 3 -n 3
pdf.groupby(['id1']).agg({'v1':'sum'})

In [ ]:
%%timeit -r 3 -n 3
cdf.groupby(['id1']).agg({'v1':'sum'})

Benchmark 2

In [ ]:
%%timeit -r 3 -n 3
pdf.groupby(['id1','id2']).agg({'v1':'sum'})

In [ ]:
%%timeit -r 3 -n 3
ddf.groupby(["id1", "id2"]).agg({'v1': 'sum'}).compute()

In [ ]:
%%timeit -r 3 -n 3
cdf.groupby(['id1','id2']).agg({'v1':'sum'})

Benchmark 3

In [ ]:
%%timeit -r 3 -n 3
pdf.groupby(['id3']).agg({'v1':'sum', 'v3':'mean'})

In [ ]:
%%timeit -r 3 -n 3
cdf.groupby(['id3']).agg({'v1':'sum', 'v3':'mean'})

Benchmark 4

In [ ]:
%%timeit -r 3 -n 3
pdf.groupby(['id4']).agg({'v1':'mean', 'v2':'mean', 'v3':'mean'})

In [ ]:
%%timeit -r 3 -n 3
cdf.groupby(['id4']).agg({'v1':'mean', 'v2':'mean', 'v3':'mean'})

Benchmark 5

In [ ]:
%%timeit -r 3 -n 3
pdf.groupby(['id6']).agg({'v1':'sum', 'v2':'sum', 'v3':'sum'})

In [ ]:
%%timeit -r 3 -n 3
cdf.groupby(['id6']).agg({'v1':'sum', 'v2':'sum', 'v3':'sum'})